# Persistent Homology of Collocations and Multiword Expressions Across Contexts for Hebrew

This notebook is for comparing how well two models preserves the persistent homology of some keyphrase like a collocation, multiword expression, or idiom. We take such a keyphrase and place it in several different contexts `text[i]`, then compute the persistent homology of the context vectors in each context. Once this is complete, we compute the pairwise distances between the persistent diagrams using the Wasserstein distance metric. This gives two distance matrices, one for each mode (here we use `xlm-roberta-large` and `TurkuNLP/wikibert-base-he-cased`). Once we have computed this distance matrix, we do a simple elementwise comparison to see what percentage of the elements in the difference of the two distance matrices are negative. Note that this is a per attention head comparison, so we can compare different attention heads in each model. Changing the second function `compute_output_b()` to use the first model `xlm-roberta-large` allows for comparing different attention heads *within the same model*. 

In [6]:
pip install transformers torch numpy gudhi -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [7]:
text = ["בחופשה האחרונה שלנו, נסענו להכיר את היופיים של מדבר הנגב. בין אם מדובר בצוקים המרשימים, בחי הבר המיוחד, או בשקט המוחלט, יש משהו מאוד מיוחד במדבר. אחת החוויות המרגשות ביותר שלנו הייתה לצפות בזרחת השמש מעל המדבר. האור המתפשט מאחורי ההרים, השמים המשתנים מאוד מהיר מאופל לתכלת, והשלווה המוחלטת שאפשר רק במדבר, הכל הפך את החוויה לבלתי נשכחת", 
          
          "במהלך שנת הלימודים הראשונה שלי באוניברסיטה, הצטרפתי לקבוצת טיול שהגיעה להר האייפל. למרות הקור החודר, הייתי מחויב לעלות לפסגה בכל בוקר, כדי לצפות בזרחת השמש מעל פריס", 
          
          "הייתי מתעורר בבוקר מוקדם, לפני כולם, כדי לצפות בזרחת השמש. היו ימים שהשמים היו מלאים בגוונים של ורוד וכתום, והאוויר הקר היה ממלא את הריאות. הייתי נושם את השקט, מאזין לשירת הציפורים, ומרגיש את היום החדש שמתחיל.", 
          
          "הייתה לי הרגל לצפות בזרחת השמש כאשר הייתי טסה למקומות רחוקים. הייתי מתמקדת באור הזהב של השמש שהתפשט על פני האופק. זה היה רגע של שקט ושלווה, שבו הייתי מרגישה את האפשרויות של היום שלפני.", 
          
          "אחד הדברים שאני ממש אוהב לעשות בחופשות הוא לצפות בזרחת השמש על גג המלון. אין דבר יותר מרגיע מאשר לשבת עם כוס קפה ביד, להתבונן בנוף, ולראות איך העולם מתעורר לחיים.", 
          
          "במהלך ההליכה, עצרנו לרגע כדי לצפות בזרחת השמש. האור הראשוני של היום הזהיר את השמיים בצבעים של זהב, ואנחנו ישבנו שם בשקט, מתפללים ליום טוב.", 
          
          "אני מאמין שאין דבר מרגש יותר מ לצפות בזרחת השמש. כאשר האור הראשונים מתחילים להתפשט באופק, אתה מרגיש כאילו אתה חלק ממשהו גדול מאוד. זה מזכיר לי כמה העולם הזה גדול ויפה.", 
          
          "אחת הפעמים המיוחדות ביותר שבהן הזמנתי לצפות בזרחת השמש הייתה בחופשה שלי בהודו. הייתי מתעורר מוקדם, לפני כל העולם, ומשתקף למראה המרהיבה של השמש המתעלה מעל האוקיינוס. זה היה חוויה שאני לעולם לא אשכח.", 
          
          "אני אוהב לצפות בזרחת השמש מהחלון שלי. זה נותן לי את האנרגיה להתחיל את היום. אני אפילו מקדיש כמה דקות בכל בוקר לקחת כוס קפה, לשבת מול החלון, ולהשתקף במראה המדהים הזה.", 
          
          "בראשית, אני אוהב לצפות בזרחת השמש מהמרפסת שלי. זה מזכיר לי את היופי של העולם, את התקווה של יום חדש, ואת החיים הממשיכים להתפתח בכל יום. כל זריחה מציגה תמונה שונה, נוף חדש שממלא אותי בתחושת התרגשות והתפעלות.", 
          
          "היום האחרון שלי ביפן היה יום מיוחד. יצאתי להליך מוקדם בבוקר, כדי לצפות בזרחת השמש מעל הר הפוג'. האור הראשוני של היום מאיר את השיחים המקופים בשלג, מצייר תמונה יפהפיה שאני לעולם לא אשכח.", 
          
          "אחד החוויות המרגשות ביותר שלי היתה לצפות בזרחת השמש מעל הפירמידות במצרים. האור החום החודר את השחקים, מאיר את האבנים העתיקות, ומעניק להם מראה של זהב. זה היה רגע של התבוננות והתפעלות על ההיסטוריה שלנו.", 
          
          "כאשר אני מטייל בים, אני מתכנן לצפות בזרחת השמש מהחוף. אין דבר מרהיב יותר מלראות את האור הראשון של היום מתפשט על גלי הים, משנה את צבעם לגוונים של זהב ואורנג'. זה הופך את החוויה של ההליכה למשהו יוצא דופן.", 
          
          "האלפים היה חלום שלי. כשהגעתי לשם, הייתי עייף אבל מרוצה. לצפות בזרחת השמש מהפסגה, כשהאור העדין של השחר התחיל להתפשט על השפעי השלג הלבנים, היה חוויה בלתי נשכחת. העולם התמלא בנופים שלא ראיתי מעולם. זה היה מרגע של שלווה ושקט, שהיה שווה את כל המאמץ.", 
          
          "אחרי שהוא התעורר מהשנה, איזיק הכין לעצמו כוס קפה והולך להסתובב בגן הפרטי שלו. זו הייתה הדרך האהובה עליו להתחיל את היום - לצפות בזרחת השמש ולשמוע את הציפורים מצפצפות.", 
          
          "את חייבת לצפות בזרחת השמש מהחוף שלנו, אמר יגאל למרים, כאשר הם הגיעו לבית הנופש של המשפחה. זו תחוויה בלתי נשכחת, משהו שתזכורי לעוד שנים.",
          
          "בזמן שהכל בעיר עדיין ישן, רבקה מתעוררת בשעה המוקדמת ביותר שאפשר, מתארגנת, ויוצאת לרוץ. היא אוהבת את השקט של אותן שעות, והמראה של העיר שמתעוררת לחיים. אבל מעל כל, זה הזמן היחיד שהיא יכולה לצפות בזרחת השמש בלי להיות מופרעת.",

          "לשפת האגם הגיעה מיה, המצלמה שלה כבר מוכנה לפעולה. היא התיישבה בשקט, מצפה לרגע הנכון. היא יודעת שממש בקרוב, היא תהיה מסוגלת לצפות בזרחת השמש, והיא רוצה לתפוס את הרגע המושלם בתמונה."

]


In [8]:
for i in range(len(text)):
    print(i, text[i])

0 בחופשה האחרונה שלנו, נסענו להכיר את היופיים של מדבר הנגב. בין אם מדובר בצוקים המרשימים, בחי הבר המיוחד, או בשקט המוחלט, יש משהו מאוד מיוחד במדבר. אחת החוויות המרגשות ביותר שלנו הייתה לצפות בזרחת השמש מעל המדבר. האור המתפשט מאחורי ההרים, השמים המשתנים מאוד מהיר מאופל לתכלת, והשלווה המוחלטת שאפשר רק במדבר, הכל הפך את החוויה לבלתי נשכחת
1 במהלך שנת הלימודים הראשונה שלי באוניברסיטה, הצטרפתי לקבוצת טיול שהגיעה להר האייפל. למרות הקור החודר, הייתי מחויב לעלות לפסגה בכל בוקר, כדי לצפות בזרחת השמש מעל פריס
2 הייתי מתעורר בבוקר מוקדם, לפני כולם, כדי לצפות בזרחת השמש. היו ימים שהשמים היו מלאים בגוונים של ורוד וכתום, והאוויר הקר היה ממלא את הריאות. הייתי נושם את השקט, מאזין לשירת הציפורים, ומרגיש את היום החדש שמתחיל.
3 הייתה לי הרגל לצפות בזרחת השמש כאשר הייתי טסה למקומות רחוקים. הייתי מתמקדת באור הזהב של השמש שהתפשט על פני האופק. זה היה רגע של שקט ושלווה, שבו הייתי מרגישה את האפשרויות של היום שלפני.
4 אחד הדברים שאני ממש אוהב לעשות בחופשות הוא לצפות בזרחת השמש על גג המלון. אין דבר יותר מרגיע מא

In [9]:
import torch
from transformers import BertTokenizer, BertModel
from transformers import BertModel, BertTokenizerFast
from transformers import AutoTokenizer, AutoModel  # Corrected import here

def compute_output(sentence, layer, head):
    # Load pre-trained model
    tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
    model = AutoModel.from_pretrained("xlm-roberta-large", output_attentions=True) 

    # Tokenize input and convert to tensor
    inputs = tokenizer(sentence, return_tensors="pt")

    # Forward pass
    # Specify `output_hidden_states=True` when calling the model
    outputs = model(**inputs, output_attentions=True, output_hidden_states=True)

    # Obtain the attention weights
    attentions = outputs.attentions

    # Obtain the attention weights for the specific layer and head
    S = attentions[layer][0, head]

    # Obtain the value vectors
    model.eval()
    with torch.no_grad():
        hidden_states = outputs.hidden_states[layer]
        all_W_v = model.encoder.layer[layer].attention.self.value.weight
        num_heads = model.config.num_attention_heads
        head_dim = model.config.hidden_size // num_heads
        W_v_heads = all_W_v.view(num_heads, head_dim, model.config.hidden_size)
        W_v = W_v_heads[head]
        V = torch.matmul(hidden_states, W_v.t())

    # Compute the output O
    O = torch.matmul(S, V)

    return O

In [10]:
# Set the layer and head to use for computation
layer = 7
head = 3

context = []
for i in range(len(text)):
    context.append(compute_output(text[i], layer, head))

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a

In [11]:
from scipy.spatial import distance_matrix
import gudhi as gd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer

def compute_phrase_distances_and_homology(context_vectors, sentence, phrase):
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

    # Tokenize the sentence and the phrase
    sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
    phrase_tokens = tokenizer.encode(phrase, add_special_tokens=False)

    # Find the indices of the phrase tokens in the sentence
    phrase_indices = []
    phrase_length = len(phrase_tokens)
    for i in range(len(sentence_tokens) - phrase_length + 1):
        if sentence_tokens[i:i+phrase_length] == phrase_tokens:
            phrase_indices.extend(range(i, i+phrase_length))
            break

    # Extract the context vectors for the phrase
    phrase_context_vectors = context_vectors[0, phrase_indices]

    # Detach the tensor and convert to numpy array
    phrase_context_vectors_np = phrase_context_vectors.detach().numpy()

    # Print the tokens of the sub-collection and their context vectors
    # print(f'Tokens of the sub-collection: {tokenizer.convert_ids_to_tokens(phrase_tokens)}')
    # print(f'Context vectors of the sub-collection: {phrase_context_vectors_np}')

    # Compute the pairwise Euclidean distances among the phrase context vectors
    distances = distance_matrix(phrase_context_vectors_np, phrase_context_vectors_np)

    # Print the distance matrix
    # print(f'Distance matrix: {distances.shape}')
    # print(f'Distance matrix: {distances}')

    # Compute the persistent homology of the distance matrix
    rips_complex = gd.RipsComplex(distance_matrix=distances, max_edge_length=np.max(distances))
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
    persistent_homology = simplex_tree.persistence(min_persistence=0.001)

    # Plot the barcode diagram
    # gd.plot_persistence_barcode(persistence=persistent_homology)
    # plt.show()

    return persistent_homology

In [12]:
persistent_homology = []
for i in range(len(text)):
    persistent_homology.append(compute_phrase_distances_and_homology(context[i], text[i], "לצפות בזרחת השמש"))

In [13]:
from gudhi.hera import wasserstein_distance
import numpy as np

def transform_persistence_diagram(diagram):
    # Remove the dimension from each feature and return the transformed diagram
    return [(birth, death) for dimension, (birth, death) in diagram]

def compute_wasserstein_distances(persistence_diagrams, p=2):
    n = len(persistence_diagrams)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            diagram1 = transform_persistence_diagram(persistence_diagrams[i])
            diagram2 = transform_persistence_diagram(persistence_diagrams[j])
            distance = wasserstein_distance(diagram1, diagram2, order=1., internal_p=2.)
            distances[i, j] = distance
            distances[j, i] = distance
    return distances

In [14]:
persistence_diagrams_1 = []
for i in range(len(text)):
    persistence_diagrams_1.append(persistent_homology[i])

In [15]:
w_distances = compute_wasserstein_distances(persistence_diagrams_1)
print(w_distances)

[[0.         0.14358496 0.70826325 0.4711087  0.46510486 0.66519472
  0.72106172 0.64639516 0.25645569 0.74026037 0.57935826 0.66323981
  0.65809475 0.50263554 0.33131627 0.75071186 0.62353368 0.6456653 ]
 [0.14358496 0.         0.6067536  0.38272286 0.46688104 0.52160976
  0.63267588 0.53151107 0.30987864 0.78782533 0.50614469 0.68760714
  0.65631857 0.47948699 0.36945366 0.66232602 0.58357096 0.64388911]
 [0.70826325 0.6067536  0.         0.51532513 0.87037348 0.35806367
  0.24278376 0.17987196 0.63386834 0.35499558 0.128905   1.15788785
  0.17214855 0.39345292 0.62027882 0.42893342 0.15904181 0.19465545]
 [0.4711087  0.38272286 0.51532513 0.         0.46565468 0.25949316
  0.34015762 0.37705579 0.39888494 0.73971055 0.48912666 0.89639451
  0.65414876 0.35109556 0.2456747  0.30970259 0.4746897  0.64511548]
 [0.46510486 0.46688104 0.87037348 0.46565468 0.         0.66451876
  0.77979867 0.82024867 0.6845074  1.19196901 0.91484238 0.82375536
  1.00976632 0.79432513 0.57246977 0.5936490

---

In [16]:
import transformers
import torch
from transformers import BertTokenizer, BertModel


def compute_output_b(sentence, layer, head):
    # Load pre-trained model
    tokenizer = transformers.BertTokenizer.from_pretrained("TurkuNLP/wikibert-base-he-cased")
    model = transformers.BertModel.from_pretrained("TurkuNLP/wikibert-base-he-cased", output_attentions=True)


    # Tokenize input and convert to tensor
    inputs = tokenizer(sentence, return_tensors="pt")

    # Forward pass
    # Specify `output_hidden_states=True` when calling the model
    outputs = model(**inputs, output_attentions=True, output_hidden_states=True)

    # Obtain the attention weights
    attentions = outputs.attentions

    # Obtain the attention weights for the specific layer and head
    S = attentions[layer][0, head]

    # Obtain the value vectors
    model.eval()
    with torch.no_grad():
        hidden_states = outputs.hidden_states[layer]
        all_W_v = model.encoder.layer[layer].attention.self.value.weight
        num_heads = model.config.num_attention_heads
        head_dim = model.config.hidden_size // num_heads
        W_v_heads = all_W_v.view(num_heads, head_dim, model.config.hidden_size)
        W_v = W_v_heads[head]
        V = torch.matmul(hidden_states, W_v.t())

    # Compute the output O
    O = torch.matmul(S, V)

    return O

In [17]:
# Set the layer and head to use for computation
layer = 7
head = 3

context_b = []
for i in range(len(text)):
    context_b.append(compute_output_b(text[i], layer, head))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of the model checkpoint at TurkuNLP/wikibert-base-he-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

In [18]:
persistent_homology_b = []
for i in range(len(text)):
    persistent_homology_b.append(compute_phrase_distances_and_homology(context_b[i], text[i], "לצפות בזרחת השמש"))

In [19]:
persistence_diagrams_1b = []
for i in range(len(text)):
    persistence_diagrams_1b.append(persistent_homology_b[i])

In [20]:
w_distances_b = compute_wasserstein_distances(persistence_diagrams_1b)
print(w_distances_b)

[[0.         3.31376767 4.44075047 2.02051601 1.35515904 4.79234373
  5.32450433 1.26935452 3.58035912 1.58135222 4.64680879 2.67487106
  2.9142429  2.92964845 3.66830721 1.87328782 5.36017682 6.99961064]
 [3.31376767 0.         1.17177497 1.92164634 4.08763647 2.52107036
  2.13508821 2.37801159 0.62744578 2.25045337 2.66758588 1.12751091
  1.94931114 4.20132851 1.14733194 4.77353281 2.09606084 4.58255541]
 [4.44075047 1.17177497 0.         2.49276498 5.21461927 1.34929538
  1.26917333 3.22743344 1.21563762 2.93192877 2.29844461 1.83840993
  1.89959226 5.06724339 0.84497378 5.90051561 0.92428586 3.41078043]
 [2.02051601 1.92164634 2.49276498 0.         3.10089838 2.77182772
  3.45810769 1.07400481 2.00414379 0.95961259 4.22677255 1.57347167
  1.3273479  4.43967143 2.60621812 3.65760146 3.383868   4.97909463]
 [1.35515904 4.08763647 5.21461927 3.10089838 0.         5.8727261
  6.09837313 2.44706886 4.35422792 2.6617346  5.75384055 3.75525343
  3.99462527 2.93117481 4.74868958 1.81762611

In [21]:
print(w_distances - w_distances_b)

[[ 0.         -3.17018271 -3.73248722 -1.54940731 -0.89005418 -4.12714901
  -4.60344261 -0.62295936 -3.32390343 -0.84109185 -4.06745053 -2.01163125
  -2.25614815 -2.42701291 -3.33699094 -1.12257596 -4.73664314 -6.35394535]
 [-3.17018271  0.         -0.56502138 -1.53892349 -3.62075542 -1.9994606
  -1.50241232 -1.84650052 -0.31756715 -1.46262804 -2.16144119 -0.43990377
  -1.29299257 -3.72184153 -0.77787828 -4.11120679 -1.51248987 -3.93866629]
 [-3.73248722 -0.56502138  0.         -1.97743985 -4.34424579 -0.99123171
  -1.02638957 -3.04756148 -0.58176929 -2.57693319 -2.16953961 -0.68052208
  -1.72744371 -4.67379047 -0.22469497 -5.47158219 -0.76524405 -3.21612499]
 [-1.54940731 -1.53892349 -1.97743985  0.         -2.63524371 -2.51233456
  -3.11795007 -0.69694902 -1.60525886 -0.21990203 -3.73764589 -0.67707716
  -0.67319914 -4.08857587 -2.36054342 -3.34789887 -2.9091783  -4.33397915]
 [-0.89005418 -3.62075542 -4.34424579 -2.63524371  0.         -5.20820734
  -5.31857446 -1.62682019 -3.669720

In [22]:
import numpy as np

def count_negative_entries_below_diagonal(matrix):
    count = 0
    total = 0
    n = len(matrix)
    for i in range(n):
        for j in range(i):
            if matrix[i][j] < 0:
                count += 1
            total += 1
    return count, total

# example matrix
matrix = w_distances - w_distances_b

negative_count, total_count = count_negative_entries_below_diagonal(matrix)
percentage = (negative_count / total_count) * 100
print("Percentage of negative entries below the diagonal: ", percentage)


Percentage of negative entries below the diagonal:  99.34640522875817


## Interpreting the results

So, as we can see, the percentage of negaive entries below the diagonal is approximately $99.35\%$, meaning $99.35\%$ of the time the Wasserstein distance between persistence diagrams for the phrase "לצפות בזרחת השמש" in various different contexts (given by the text corpera at the beginning of the notebook) is lower for `TurkuNLP/wikibert-base-he-cased` than for `xlm-roberta-large`. This means that the `TurkuNLP/wikibert-base-he-cased` model preserves the persistent homology for the context vectors associated to the phrase "לצפות בזרחת השמש" better in more contexts. Note, this is comparing `head 3` of `layer 7` in the models. When we compare different heads this number will change and some heads drastically outperform others at preserving persistent homology of keyphrases. 

It is interesting to note that we might linguistically analyze a model's individual attention heads using this method to determine what kinds of keyphrases it preserves. We might also use this to modify attention heads to better understand certain keyphrases. This is closely linked to topic modeling as is mentioned in [Topics in Contextualised Attention Embeddings](https://arxiv.org/pdf/2301.04339.pdf). This, along with the Fréchet mean of persistence diagrams could also be used for anomaly detection, where an anomaly is considered a "significant" deviation from the Fréchet mean persistence diagram. For character level transformers this might also be used as a topological prior to obtain something like hierarchical morphological segmentations of words, where the hierarchy is given by the simplex tree, and the simplex tree is encouraged to mimic heirarchical morphological segmentation of words similar to what is described in [Morphological Segmentation Inside-Out](https://arxiv.org/abs/1911.04916v2). 